In [1]:
import codecs, nltk, string, os, gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


exclude = set(string.punctuation)

# this represent any text as a single "doc-embedding" we use it both for the query and the sentences
# input should be a string
def text_embedding(text):
    
    #you should check in the embeddings you use if the words have been lowercased or not. 
    #try ask the embedding for "barack" and for "Barack"
    # if the Barack works, then comment the following line
    text = text.lower()
    
    # we tokenize the text in single words
    text = nltk.tokenize.WordPunctTokenizer().tokenize(text)
    
    # we remove numbers and punctuation
    text = [token for token in text if token not in exclude and token.isalpha()]
    
    
    doc_embed = []
    
    # for each word we get the embedding and we append it to a list
    for word in text:
            try:
                embed_word = emb_model[word]
                doc_embed.append(embed_word)
            except KeyError:
                continue
    # we average the embeddings of all the words, getting an overall doc embedding
    if len(doc_embed)>0:
        avg = [float(sum(col))/len(col) for col in zip(*doc_embed)]

        avg = np.array(avg).reshape(1, -1)

        # the output is a doc-embedding
        return avg
    else:
        return "Empty"

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#import here your word-embeddings - put the path to the file (if it's .bin change the binary to True)
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('../../resources/small-embeddings.txt', binary=False)

# german wikipedia from https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/federiconanni/Downloads/wiki.de.vec', binary=False)
emb_model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Dr. J/Desktop/wiki.de.vec', binary=False)
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Dr. J/Desktop/wiki.en.vec', binary=False)

# switch for in-domain 
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('C:/QTA/topic-detect/in-domain-embeddings.txt', binary=False)



In [3]:
#this can be a list of words on the same fine-grained topic, like "people", "elites"
# add more words after a space to make it more precise
# query = "volk bürger"

#focus on "key key" terms based on dictionary, let the embeddings find more 
#query = "volk"
#query = "bürger einbürgerung steuerzahler gemeinschaft"
#query = "elite politiker establishment herrschend"
#query = "korruption täuschung betrügen verrat schämen skandal wahrheit unehrlich lüge"
#query = "verantwortung glaubwürdigkeit"
#query = "souverän neutral"
#query = "demokratisch referendum volksabstimmung volksinitiative"
#query = "konsens kompromiss"
#query = "repräsentation parlament regierung "
#query = "populisten populismus demagogisch demagogen"

# populism at its best?
query = "volk elite souverän"

# efficient flip-side query?
#query = "konsens repräsentation populismus"

# query = "volk elite souverän parlament regierung repräsentation"

#query = "people elite sovereign"

query_emb = text_embedding(query)

# add the path to the folder where you have your manifestos as text documents
# collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/deu2017/"

# only Germany for less output 
#collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/germany17/"

# German texts last three elections for Germany, Austria and Switzerland - 6X manifestos from polidoc.net
collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/polidoc_allgerman/"
#collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/en/"

In [4]:
# this will be a dictionary of documents, for example manifestos, divided in sentences, which are represented as sentence embeddings
collection = {}

# you loop over the folder
for filename in os.listdir(collection_path):
    # you open each file
    # note encoding 
    content = codecs.open(collection_path+filename,"r","utf-8").read()
    # you split it in sentences
    content = nltk.sent_tokenize(content)
    
    # you represent each sentence in each document as a word-embedding, which captures the meaning of the sentence
    content = [[sent, text_embedding(sent)] for sent in content if type(text_embedding(sent))!= str]
    collection[filename] = content

In [5]:
# now, the information retrieval part

for filename,sentences in collection.items():
    
    # compare the cosine similarity between the embedding of the query and each sentence embedding
    ranking = [[sent, cosine_similarity(query_emb,sent_emb)[0][0]] for sent, sent_emb in sentences]
    # you rank them, based on the similarity
    ranking.sort(key=lambda x: x[1],reverse=True)
    
    print (filename)
    # you can change here for having more sentences as output
   # for sent, score in ranking[:20]:
    for sent, score in ranking[:3]:
        print (sent, score)
    print (" \n")

41113.000.2009.1.1.txt
Anwältin der Bürgerrechte – Rechtsstaat verteidigen, Integration fördern
2009 blicken wir zurück auf 60 Jahre Grundgesetz, auf 20 Jahre friedliche Revolution in der
DDR, auf 90 Jahre Frauenwahlrecht. 0.5863941588630321
Das Grundgesetz garantiert auch ihnen im Rahmen der freiheitlich-demokratischen Grundordnung
gleichberechtigt individuelle und kollektive Religionsfreiheit. 0.5857661408278807
Populistische Ressentiments gegen Parlamente und Parteien sind gefährlich
für unsere Demokratie. 0.5751424319890812
 

41113.000.2013.1.1.txt
Die Demokratie vervollständigen
Unsere Demokratie lebt vom Engagement, vom Mitmachen. 0.610229959373757
Unsere Demokratie braucht einen selbstbewussten Bundestag mit starken, unabhängigen Abgeordneten, die die Gesellschaft abbilden und dem Drängen mächtiger Interessen standhalten. 0.5896222846464771
Parlamente stärken, Parteien öffnen
Die repräsentative Demokratie ist Garant für die Vertretung der gesamten Gesellschaft gegenüber Einzeli

41950.000.2013.1.1.txt
Chancengleichheit im Bildungssystem
Die Piratenpartei setzt sich für mehr Chancengleichheit ein: Die sozio-kulturelle Herkunft darf nicht mehr über den Bildungserfolg entscheiden. 0.5786036743214833
Verteidigung der freiheitlich-demokratischen Grundordnung
und des Rechtsstaats
Nicht abgeschafft werden sollen dagegen solche Verbote, die notwendig sind, um unsere freiheitlich-demokratische Grundordnung und den Rechtsstaat zu verteidigen. 0.5701796472386296
Die Bürger sollen in Volksabstimmungen direkt über Gesetze entscheiden können und so Verantwortung übernehmen:
- Ein im Parlament beschlossenes Gesetz soll nicht in Kraft treten, wenn die Bürger es in einer Abstimmung ablehnen (fakultatives Referendum). 0.5640221896195254
 

41953.000.2013.1.1.txt
Jedes Volk muss
demokratisch über seine Währung entscheiden dürfen. 0.668985516819088
Das Volk
soll den Willen der Parteien bestimmen, nicht
umgekehrt. 0.6056140002852074
Rechtsstaatlichkeit und Demokratie
- Wir fordern

42450.000.2013.1.1.txt
Unsere Kernwerte
- freiheitsliebend
Mündig
Unabhängig
Entscheidungsfrei
Flexibel
Aufgeschlossen
- wertschätzend
Fair
Gemeinsam gestalten
Europäisch und weltoffen
Sozial verantwortlich und integrativ
Leistungsorientiert
- nachhaltig
Chancengerecht
Integraler Ansatz
Langfristorientiert
Zukunftsfreude statt Zukunftsangst
Balance von Gesellschaft, Wirtschaft und Ökologie
- authentisch
Überlegt
Kompetent und lernfähig
Aufrichtig und echt
Transparent
Klar und verständlich
- eigenverantwortlich
verantwortungsbewusst
ergebnisorientiert
entschlossen und mutig
vernünftig und pragmatisch
Bürgerrechte und Demokratie stärken


DEMOKRATIE
DIE HERAUSFORDERUNG
Österreichs Staats- und Verwaltungsstruktur ist höchst ineffizient und längst
nicht mehr zeitgemäß. 0.5804618971360878
Aus der
multikulturellen Gesellschaft vor 1918 verblieb ein Land, das seine Identität als
deutsch sprechender Kleinstaat lange suchen musste und erst nach 1945 ein
eigenständiges Nationalgefühl entwickelte

43520.000.2015.1.1.txt
Bei uns regiert das Volk. 0.7024402303188635
Wir unterstützen die erleichterte Einbürgerung junger Ausländerinnen und Ausländer, die hier in die Schule gegangen sind, unsere Sprache sprechen und gut integriert sind. 0.5458683893545874
Dank der CVP kann sich das Volk endlich zu dieser Frage äussern. 0.5452945078220958
 

43530.000.2007.1.1.txt
Darum unterstützt der Staat die Familien. 0.5179779762868801
Weiter verstehen wir darunter eine gerechte Gesellschaft und eine tragfähige Gemeinschaft von Menschen, die von gegenseitigem Vertrauen und Rücksichtnahme geprägt ist und die niemanden ausschliesst. 0.5043711658802339
Gerechte Wirtschaft dient dem Wohl aller
Der Staat soll die Rahmenbedingungen für eine gut funktionierende, gerechte Marktwirtschaft schaffen. 0.5009929933046989
 

43530.000.2011.1.1.txt
Beispiel: der Islam hat in Europa nur dann seinen Platz, wenn er den Pluralismus der Religionen und Kulturen anerkennt, d.h. seinen Dominanzanspruch auf Überlegenhei